Board size: 12x10 -- any coor that falls outside of the valid cells is considered offboard

Cell encoding:
0   1   2   3   4   5   6   7   8   9   
10  11  12  13  14  15  16  17  18  19  
20  21  22  23  24  25  26  27  28  29  
30  31  32  33  34  35  36  37  38  39  
40  41  42  43  44  45  46  47  48  49  
50  51  52  53  54  55  56  57  58  59  
60  61  62  63  64  65  66  67  68  69  
70  71  72  73  74  75  76  77  78  79  
80  81  82  83  84  85  86  87  88  89  
90  91  92  93  94  95  96  97  98  99  
100 101 102 103 104 105 106 107 108 109 
110 111 112 113 114 115 116 117 118 119 

Valid cells:
   [21, 22, 23, 24, 25, 26, 27, 28, 
    31, 32, 33, 34, 35, 36, 37, 38, 
    41, 42, 43, 44, 45, 46, 47, 48, 
    51, 52, 53, 54, 55, 56, 57, 58, 
    61, 62, 63, 64, 65, 66, 67, 68, 
    71, 72, 73, 74, 75, 76, 77, 78, 
    81, 82, 83, 84, 85, 86, 87, 88, 
    91, 92, 93, 94, 95, 96, 97, 98]

Direction encoding:
    up : -10
    down : 10
    right : 1
    left : -1
    up_left : -11
    up_right : -9
    down_left : 9
    down_right : 11
    knight movements clockwise starting up_up_left : [-21, -19, -8, 12, 21, 19, 8, -12]

Piece types:
    slider: bishop, rook, queen
    leaper: knight, king
    pawn

Piece encoding:
    empty : 0
    pawn : 1
    knight : 2
    bishop : 3
    rook : 4
    queen : 5
    king : 6
    --> if black piece * -1

Board: dict with key=cell_encoding, value=piece_code +++ castle, side, xside


cell//10==8 if side=1 and cell//10==3 of side=-1

In [1]:
from time import perf_counter as ttt
import numpy as np

from done.get import get_occupant
from done.get import get_piece
from done.indicator import is_opponent
from done.indicator import is_traversible
from done.prelegal import pre_pawn
from done.prelegal import pre_leaper
from done.prelegal import pre_slider

In [5]:
# cell:piece
with open('done/vars/ini_board.txt', 'r') as f:
    ini_board = eval(f.read())

with open('done/vars/valid_cells.txt', 'r') as f:
    valid_cells = eval(f.read())

In [4]:
def display_stats(time_log, num_runs):
    print(f'Number of runs for all functions: {num_runs}')
    for func_name, dur_hist in time_log.items():
        # function name
        print(f'{func_name.upper()}')
        # total time and average time
        total_time = sum(dur_hist)
        print(f'Total time: {total_time}s\nAverage time per run: {total_time/num_runs/1000}ms')
        # median time
        ord_hist = sorted(dur_hist)  # sort the duration log in ascending order
        if len(ord_hist)%2 == 0:
            median = (ord_hist[int(len(ord_hist)/2)] + ord_hist[int(len(ord_hist)/2-1)])/2
        else:
            median = ord_hist[int(len(ord_hist)/2-0.5)]
        print(f'Median Time taken for {n_iter} runs: {median}s')

In [ ]:
with open('done/vars/move_dirs.txt', 'r'):
    move_dirs = eval(f.read())

# try with final_prelegal already prepped with all keys, to do is_check see if prelegal[king_cell] == []

def pre_board1(board):
    final_prelegal = {}
    piece_to_prelegal_func = {1:pre_pawn, 2:pre_leaper, 3:pre_slider, 4:pre_slider, 5:pre_slider, 6:pre_leaper}
    for piece_cell, piece_index in board.items():
        func = piece_to_prelegal_func[piece_index]
        if piece_index == 1:
            prelegal_piece = func(board, piece_cell, np.sign(piece_index))
        else:
            prelegal_piece = func(board, piece_cell, move_dirs[piece_index], np.sign(piece_index))

        for dest in prelegal_piece:
            try:
                final_prelegal[dest].append(piece_cell)
            except:
                final_prelegal[dest] = [piece_cell]
    return final_prelegal


def pre_board2(board):
    final_prelegal = {}
    piece_to_prelegal_func = {1:pre_pawn, 2:pre_leaper, 3:pre_slider, 4:pre_slider, 5:pre_slider, 6:pre_leaper}
    for piece_cell, piece_index in board.items():
        func = piece_to_prelegal_func[piece_index]
        if piece_index == 1:
            prelegal_piece = func(board, piece_cell, np.sign(piece_index))
        else:
            prelegal_piece = func(board, piece_cell, move_dirs[piece_index], np.sign(piece_index))

        for dest in prelegal_piece:
            try:
                final_prelegal[dest] += [piece_cell]
            except:
                final_prelegal[dest] = [piece_cell]
    return final_prelegal

# i want to find if king_cell is inside the end_cell of any prelegal move (to find if in check)
# prelegal_dict = {end_cell:[start_cell1, start_cell2]} -> dict[end_cell] += [start_cell]

# tuple move representation (start_cell, end_cell)

# i can either directly have list of prelegal_moves in tuple form, or have dict of prelegal_moves
# dict is easier to search for king, but extra conversion step when generating list of legal moves

# is there a way to transfer info from one board to another board? might be quicker to only recalculate certain pieces prelegal moves
# can i detect if a piece is in a slider postion from king? pieces in knight positions must be legal to move(?)

# check range of motion of queen from king_cell, if the first enemy along that line threatens the king, and there is only one ally along that path, then the ally cannot move
# is reveal check the only way for a move to be prelegal but not legal????? check this further

2 possibilities for move representation: 

dict = {end_cell: [start_cell1, start_cell2]}

tuple = (start_cell, end_cell)

dict will make is_check faster

tuple more readable

how to use dict in further steps?

final output of legal_moves should be list of tuples/array


2 ways for non-legality

king is already under check and move does not fix that
king is pinned and middle piece moves away

approach 1: psuedomove every single piece and check if that move places the king under check

approach 2: only check legality of pieces that are in slider coors wrt king (if check, pseudomove all pieces)

# Speed Test

In [ ]:
### GET OCCUPANT ###

def get_occupant1(board, cell):
    if cell in valid_cells:
        if cell in board:
            return np.sign(board[cell])
        else:
            return 0

def get_occupant2(board, cell):
    try:
        return np.sign(get_piece)
    except:
        return None


n_iter = 10000
test_cells = range(100)

start = ttt()
for _ in range(n_iter):
    for cell in test_cells:
        get_occupant1(ini_board, cell)
end = ttt()

print(f'get_occupant1 takes time {end-start} to run {n_iter*len(test_cells)} iterations. Average time per run: {(end-start)/n_iter/len(test_cells)/1000}ms')

start = ttt()
for _ in range(n_iter):
    for cell in test_cells:
        get_occupant2(ini_board, cell)
end = ttt()

print(f'get_occupant2 takes time {end-start} to run {n_iter*len(test_cells)} iterations. Average time per run: {(end-start)/n_iter/len(test_cells)/1000}ms')

In [ ]:
### GET PIECE ###

def get_piece1(board, cell):
    if cell in valid_cells:
        if cell in board:
            return board[cell]
        else:
            return 0

def get_piece2(board, cell):
    if cell in valid_cells:
        try:
            return board[cell]
        except:
            return 0

n_iter = 10000
test_cells = range(100)

start = ttt()
for _ in range(n_iter):
    for cell in test_cells:
        get_piece1(ini_board, cell)
end = ttt()

print(f'get_piece1 takes time {end-start} to run {n_iter*len(test_cells)} iterations. Average time per run: {(end-start)/n_iter/len(test_cells)/1000}ms')

start = ttt()
for _ in range(n_iter):
    for cell in test_cells:
        get_piece2(ini_board, cell)
end = ttt()

print(f'get_piece2 takes time {end-start} to run {n_iter*len(test_cells)} iterations. Average time per run: {(end-start)/n_iter/len(test_cells)/1000}ms')

In [ ]:
### IS_OPPONENT ###

'''Function definitions to be tested'''
def is_opponent1(board, cell, side):
    if cell in valid_cells:
        try:
            if np.sign(board[cell]) == -side:
                return True
            return False
        except:
            return False

def is_opponent2(board, cell, side):
    if cell in valid_cells:
        if cell in board:
            if np.sign(board[cell]) == -side:
                return True
            return False
        return False

def is_opponent3(board, cell, side):
    occupant = get_occupant(board, cell)
    if occupant == -side:
        return True
    return False

'''Variables to adjust'''
funcs = [is_opponent1, is_opponent2, is_opponent3]
n_iter = 1000  # repeat one function n_iter times
n_loops = 20  # loop through all functions 10 times
test_cells = range(100)
sides = [1, -1]


'''Precalculated variables'''
total_runs = n_loops*n_iter*len(test_cells)*len(sides)
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for cell in test_cells:
                for side in sides:
                    func(ini_board, cell, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]




'''Display time_log statistics'''
display_stats(time_log, total_runs)

In [ ]:
### MATH FUNCTION VS IF STATEMENT ###

'''Function definitions to be tested'''
def test1(cell, side):
     return cell//10==side*2.5+5.5

def test2(cell, side):
    x = cell//10==8 if side == 1 else cell//10==3
    return x

'''Variables to adjust'''
funcs = [test1, test2]
n_iter = 1000  # repeat one function n_iter times
n_loops = 100  # loop through all functions 10 times
test_cells = range(100)
sides = [1, -1]


'''Precalculated variables'''
total_runs = n_loops*n_iter*len(test_cells)*len(sides)
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for cell in test_cells:
                for side in sides:
                    func(cell, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

In [ ]:
### PRELEGAL PAWN ###

'''Function definitions to be tested'''
def prelegal_pawn1(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3
    # generate occupancy of each new_cell in a list
    occupancy = [done.get_occupant(board, i) for i in new_cell]
    # create a mask with different conditions for different items in occupancy and new_cell list
    mask = [occupancy[0]==0, occupancy[1]==0 and jump_allowed, occupancy[2]==-side, occupancy[3]==-side]

    # loop through both mask and new_cell and append the new_cell values which have corresponding True mask value. Avoids using np
    output = []
    for mask_val, cell in zip(mask, new_cell):
        if mask_val == True:
            output.append(cell)
    return output

def prelegal_pawn2(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3

    # generate occupancy of each new_cell in a list
    occupancy = [done.get_occupant(board, i) for i in new_cell]
    # create a mask with different conditions for different items in occupancy and new_cell list
    mask = [occupancy[0]==0, occupancy[1]==0 and jump_allowed, occupancy[2]==-side, occupancy[3]==-side]
    # convert new_cell to np.ndarray and use mask to select the items which fulfill conditions
    return np.array(new_cell)[mask]

def prelegal_pawn3(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    # calculate if first move jump is allowed for this pawn_cell and side
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3

    # go through each condition iteratively and append to final output list
    output = []
    if done.get_occupant(board, new_cell[0]) == 0:
        output.append(new_cell[0])
    if done.get_occupant(board, new_cell[1]) == 0 and jump_allowed:
        output.append(new_cell[1])
    if done.get_occupant(board, new_cell[2]) == -side:
        output.append(new_cell[2])
    if done.get_occupant(board, new_cell[3]) == -side:
        output.append(new_cell[3])

    return output

def prelegal_pawn4(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3
    
    # use mask with is_opponent()
    mask = [done.get_occupant(board, new_cell[0])==0, done.get_occupant(board, new_cell[1])==0 and jump_allowed, done.is_opponent(board, new_cell[2], side), done.is_opponent(board, new_cell[3], side)]
    
    # loop through both mask and new_cell and append the new_cell values which have corresponding True mask value. Avoids using np
    output = []
    for mask_val, cell in zip(mask, new_cell):
        if mask_val == True:
            output.append(cell)
    return output

def prelegal_pawn5(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3
    
    # use mask with is_opponent()
    mask = [done.get_occupant(board, new_cell[0])==0, done.get_occupant(board, new_cell[1])==0 and jump_allowed, done.is_opponent(board, new_cell[2], side), done.is_opponent(board, new_cell[3], side)]
    
    # convert new_cell to np.ndarray and use mask to select the items which fulfill conditions
    return np.array(new_cell)[mask]

def prelegal_pawn6(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3

    # go through each condition iteratively and append to final output list
    output = []
    if done.get_occupant(board, new_cell[0]) == 0:
        output.append(new_cell[0])
    if done.get_occupant(board, new_cell[1]) == 0 and jump_allowed:
        output.append(new_cell[1])
    if done.is_opponent(board, new_cell[2], side) == -side:
        output.append(new_cell[2])
    if done.is_opponent(board, new_cell[3], side) == -side:
        output.append(new_cell[3])

    return output

'''Variables to adjust'''
funcs = [prelegal_pawn1, prelegal_pawn2, prelegal_pawn3, prelegal_pawn4, prelegal_pawn5, prelegal_pawn6]
n_iter = 1000  # repeat one function n_iter times
n_loops = 20  # loop through all functions 10 times
test_cells = range(100)
sides = [1, -1]


'''Precalculated variables'''
total_runs = n_loops*n_iter*len(test_cells)*len(sides)
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for cell in test_cells:
                for side in sides:
                    func(ini_board, cell, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

In [ ]:
### PRELEGAL PAWN WITH MAP ###

'''Function definitions to be tested'''
def prelegal_pawn3v1(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = [pawn_cell+move for move in move_dirs]
    # calculate if first move jump is allowed for this pawn_cell and side
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3

    # go through each condition iteratively and append to final output list
    output = []
    if done.get_occupant(board, new_cell[0]) == 0:
        output.append(new_cell[0])
    if done.get_occupant(board, new_cell[1]) == 0 and jump_allowed:
        output.append(new_cell[1])
    if done.get_occupant(board, new_cell[2]) == -side:
        output.append(new_cell[2])
    if done.get_occupant(board, new_cell[3]) == -side:
        output.append(new_cell[3])

    return output

def prelegal_pawn3v2(board, pawn_cell, side):
    move_dirs = [-10*side, -20*side, -11*side, -9*side]
    new_cell = list(map(lambda x: x+pawn_cell, move_dirs))
    # calculate if first move jump is allowed for this pawn_cell and side
    jump_allowed = pawn_cell//10==8 if side == 1 else pawn_cell//10==3

    # go through each condition iteratively and append to final output list
    output = []
    if done.get_occupant(board, new_cell[0]) == 0:
        output.append(new_cell[0])
    if done.get_occupant(board, new_cell[1]) == 0 and jump_allowed:
        output.append(new_cell[1])
    if done.get_occupant(board, new_cell[2]) == -side:
        output.append(new_cell[2])
    if done.get_occupant(board, new_cell[3]) == -side:
        output.append(new_cell[3])

    return output


'''Variables to adjust'''
funcs = [prelegal_pawn3v1, prelegal_pawn3v2]
n_iter = 1000  # repeat one function n_iter times
n_loops = 20  # loop through all functions 10 times
test_cells = range(100)
sides = [1, -1]


'''Precalculated variables'''
total_runs = n_loops*n_iter*len(test_cells)*len(sides)
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for cell in test_cells:
                for side in sides:
                    func(ini_board, cell, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

In [ ]:
### PRELEGAL LEAPER ###

'''Function definitions to be tested'''

def leaper_prelegal1(board, leaper_cell, move_dirs, side):
    # single if with or
    output = []
    for move_dir in move_dirs:
        new_cell = leaper_cell + move_dir
        occupant = get_occupant(board, new_cell)
        if occupant == 0 or occupant == -side:
            output.append(new_cell)
    return output

def leaper_prelegal2(board, leaper_cell, move_dirs, side):\
    # if and elif
    output = []
    for move_dir in move_dirs:
        new_cell = leaper_cell + move_dir
        occupant = get_occupant(board, new_cell)
        if occupant == 0:
            output.append(new_cell)
        elif occupant == -side:
            output.append(new_cell)
    return output

def leaper_prelegal3(board, leaper_cell, move_dirs, side):
    # using is_traversible
    output = []
    for move_dir in move_dirs:
        new_cell = leaper_cell + move_dir
        if is_traversible(board, new_cell, side):
            output.append(new_cell)
    return output

def leaper_prelegal4(board, leaper_cell, move_dirs, side):
    # using numpy (for leaper_cell) and is_traversible
    new_cells = move_dirs + leaper_cell
    mask = [False]*len(move_dirs)
    for i in range(len(move_dirs)):
        mask[i] = is_traversible(board, new_cells[i], side)
    return new_cells[mask]


'''Variables to adjust'''
funcs = [leaper_prelegal1, leaper_prelegal2, leaper_prelegal3]
n_iter = 100  # repeat one function n_iter times
n_loops = 20  # loop through all functions
test_cells = range(100)
sides = [-1, 1]
kn_md = [-21, -19, -8, 12, 21, 19, 8, -12]
kg_md = [-11, -10, -9, -1, 1, 9, 10, 11]
dirs = [kn_md, kg_md]

total_runs = n_loops*n_iter*len(test_cells)*len(dirs)


'''Preset variables'''
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for cell in test_cells:
                for side in sides:
                    for direc in dirs:
                      func(ini_board, cell, direc, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

# using numpy slows down the whole computation

In [7]:
### PRELEGAL SLIDER ###

'''Function definitions to be tested'''

def pre_slider1(board, slider_cell, move_dirs, side):
    # loop and break when reach ally/enemy/offboard
    prelegal_dest = []
    for move_dir in move_dirs:
        for _ in range(8):
            slider_cell += move_dir
            occupant = get_occupant(board, slider_cell)
            if occupant == 0:
                prelegal_dest.append(slider_cell)
            elif occupant == -side:
                prelegal_dest.append(slider_cell)
                break
            else:
                break
    return prelegal_dest

def pre_slider2(board, slider_cell, move_dirs, side):
    # map and next() until break condition
    prelegal_dest = []
    for move_dir in move_dirs:
        # calculates the destination cells in a particular direction
        dest_cells = (slider_cell+move_dir*x for x in range(1, 9))
        # create generator to get occupant of destination cells
        occ_gen = ([cell, get_occupant(board, cell)] for cell in dest_cells)
        for _ in range(8):
            cell, occ = next(occ_gen)
            if occ == 0:
                prelegal_dest.append(cell)
            elif occ == -side:
                prelegal_dest.append(cell)
                break
            else:
                break
    return prelegal_dest


'''Variables to adjust'''
funcs = [pre_slider1, pre_slider2]
n_iter = 100  # repeat one function n_iter times
n_loops = 20  # loop through all functions
test_cells = range(100)
sides = [-1, 1]
b_md = [-11, -9, 11, 9]
r_md = [-10, 1, 10, -1]
q_md = [-11, -10, -9, 1, 11, 10, 9, -1]
dirs = [b_md, r_md, q_md]

total_runs = n_loops*n_iter*len(test_cells)*len(dirs)


'''Preset variables'''
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for cell in test_cells:
                for side in sides:
                    for direc in dirs:
                      func(ini_board, cell, direc, side)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

Number of runs for all functions: 600000
PRE_SLIDER1
Total time: 36.19938050000019s
Average time per run: 6.033230083333365e-08ms
Median Time taken for 100 runs: 1.59185774999969s
PRE_SLIDER2
Total time: 72.90395869999702s
Average time per run: 1.2150659783332838e-07ms
Median Time taken for 100 runs: 2.9927012499992998s


In [14]:
### MOVE REPRESENTATION ###

'''Function definitions to be tested'''

move1 = (24, 25)
move2 = 2425

def split1(move):
    # tuple
    return move[0], move[1]

def split2(move):
    # int
    return move//100%100, move//1%100


'''Variables to adjust'''
funcs = [split1, split2]
n_iter = 10000  # repeat one function n_iter times
n_loops = 200  # loop through all functions
move1 = (24, 25)
move2 = 2425

total_runs = n_loops*n_iter*len(test_cells)*len(dirs)


'''Preset variables'''
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        if func_name == 'split1':
            move = move1
        else:
            move = move2
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            func(move)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

Number of runs for all functions: 600000000
SPLIT1
Total time: 1.4179541999910725s
Average time per run: 2.363256999985121e-12ms
Median Time taken for 10000 runs: 0.006476999999904365s
SPLIT2
Total time: 1.8196489999936603s
Average time per run: 3.0327483333227672e-12ms
Median Time taken for 10000 runs: 0.008624399999462184s


In [6]:
### APPEND VS += [] ###

'''Function definitions to be tested'''

def test1(x, y):
    # append
    x = x.append(y)
    return x

def test2(x, y):
    # += []
    x += [y]
    return x


'''Variables to adjust'''
funcs = [test1, test2]
n_iter = 10000  # repeat one function n_iter times
n_loops = 200  # loop through all functions
test_x = [[], [1], [1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
y = 0

total_runs = n_loops*n_iter*len(test_x)


'''Preset variables'''
time_log = {}
func_dict = dict([(func.__name__, func) for func in funcs])

'''Run speed test'''
for _ in range(n_loops):
    for func_name, func in func_dict.items():
        start = ttt()
        for _ in range(n_iter):
            # edit this section
            for x in test_x:
                func(x, y)
        end = ttt()
        dur = end-start
        # log the time taken
        try:
            time_log[func_name].append(dur)
        except:
            time_log[func_name] = [dur]


'''Display time_log statistics'''
display_stats(time_log, total_runs)

Number of runs for all functions: 22000000
TEST1
Total time: 6.129984300010619s
Average time per run: 2.786356500004827e-10ms
Median Time taken for 10000 runs: 0.027050450000388082s
TEST2
Total time: 6.144081499991444s
Average time per run: 2.792764318177929e-10ms
Median Time taken for 10000 runs: 0.027750149999519635s
